In [8]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np

In [2]:
original_df = pd.read_csv('2000-2023高麗菜_data_file.csv')

# 反轉DataFrame的順序，由上而下改為由下而上
# original_df = original_df[::-1].reset_index(drop=True)

高麗菜資料總整理

In [5]:
# 新建一個DataFrame用來存儲新的資料
new_data = []
original_df['finaldate'] = original_df.apply(lambda row: datetime(row['year'], row['month'], row['day']), axis=1)

# 計算89年1月1日至當下日期的天數，並加上專屬號碼
start_date = datetime(2000, 1, 1)  # 89年1月1日的日期
for index, row in original_df.iterrows():
    current_date = row['finaldate']
    days_difference = (current_date - start_date).days + 1
    new_row = {
        'dayoffset': days_difference,
        'TransDate': row['TransDate'],
        'Crop': row['Crop'],
        'MarketName': row['Market'],
        'Upper_Price': row['Upper_Price'],
        'Middle_Price': row['Middle_Price'],
        'Lower_Price': row['Lower_Price'],
        'Avg_Price': row['Avg_Price'],
        'Trans_Quantity': row['Trans_Quantity'],
        'year': row['year'],
        'month': row['month'],
        'day': row['day']
    }
    new_data.append(new_row)

# 將新的資料轉換為DataFrame
new_df = pd.DataFrame(new_data)

# 將結果反轉回原始順序，由下而上改為由上而下
# new_df = new_df[::-1].reset_index(drop=True)

# 將結果寫入新的CSV檔案
# new_df.to_csv('new_2000_2023cabbage.csv', index=False)

In [6]:
new_df.head(5)

,dayoffset,TransDate,Crop,MarketName,Upper_Price,Middle_Price,Lower_Price,Avg_Price,Trans_Quantity,year,month,day
0,2,089/01/02,LA1 甘藍 初秋,109 台北一,11.0,8.0,5.0,8.9,"192,029",2000,1,2
1,3,089/01/03,LA1 甘藍 初秋,109 台北一,9.0,7.0,4.0,7.2,"174,270",2000,1,3
2,4,089/01/04,LA1 甘藍 初秋,109 台北一,8.0,6.0,3.0,6.7,"125,426",2000,1,4
3,5,089/01/05,LA1 甘藍 初秋,109 台北一,10.0,7.0,4.0,7.1,"119,318",2000,1,5
4,6,089/01/06,LA1 甘藍 初秋,109 台北一,9.0,7.0,5.0,6.8,"118,178",2000,1,6


In [7]:
new_df.to_csv('new_2000_2023cabbage.csv', index=False)

In [8]:
new_df.shape

(7250, 12)

# 傳換天氣資料

In [11]:
def Max_10_mintes_fix_rainfall_data(df):
    df['最大十分鐘降水量(mm)'] = df['最大十分鐘降水量(mm)'].replace('T', '0.0')
    df['最大十分鐘降水量(mm)'] = df['最大十分鐘降水量(mm)'].replace('X', '-1')
    df['最大十分鐘降水量(mm)'] = pd.to_numeric(df['最大十分鐘降水量(mm)'])
    return df

def Max_60_mintes_fix_rainfall_data(df):
    df['最大六十分鐘降水量(mm)'] = df['最大六十分鐘降水量(mm)'].replace('T', '0.0')
    df['最大六十分鐘降水量(mm)'] = df['最大六十分鐘降水量(mm)'].replace('X', '-1')
    df['最大六十分鐘降水量(mm)'] = pd.to_numeric(df['最大六十分鐘降水量(mm)'])
    return df

def A_type_evaporation(df):
    df['A型蒸發量(mm)'] = df['A型蒸發量(mm)'].replace('/', '-1')
    df['A型蒸發量(mm)'] = df['A型蒸發量(mm)'].replace('X', '-1')
    df['A型蒸發量(mm)'] = df['A型蒸發量(mm)'].replace('--', '-1')
    df['A型蒸發量(mm)'] = pd.to_numeric(df['A型蒸發量(mm)'])
    return df

def All_sky_insolation(df):
    df['全天空日射量(MJ/㎡)'] = df['全天空日射量(MJ/㎡)'].replace('/', '-1')
    df['全天空日射量(MJ/㎡)'] = df['全天空日射量(MJ/㎡)'].replace('X', '-1')
    df['全天空日射量(MJ/㎡)'] = df['全天空日射量(MJ/㎡)'].replace('--', '-1')
    df['全天空日射量(MJ/㎡)'] = pd.to_numeric(df['全天空日射量(MJ/㎡)'])
    return df

def highest_UV_index_of_the_day(df):
    df['日最高紫外線指數'] = df['日最高紫外線指數'].replace('/', '-1')
    df['日最高紫外線指數'] = df['日最高紫外線指數'].replace('X', '-1')
    df['日最高紫外線指數'] = df['日最高紫外線指數'].replace('--', '-1')
    df['日最高紫外線指數'] = pd.to_numeric(df['日最高紫外線指數'])
    return df



# 合併 yilan 全部年分資料

In [12]:
merged_data = pd.DataFrame()
column_names = ["觀測時間(day)", "測站氣壓(hPa)", "海平面氣壓(hPa)", "測站最高氣壓(hPa)", "測站最高氣壓時間(LST)", "測站最低氣壓(hPa)", "測站最低氣壓時間(LST)", "氣溫(℃)", "最高氣溫(℃)", "最高氣溫時間(LST)", "最低氣溫(℃)", "最低氣溫時間(LST)", "露點溫度(℃)", "相對溼度(%)", "最小相對溼度(%)", "最小相對溼度時間(LST)", "風速(m/s)", "風向(360degree)", "最大瞬間風(m/s)", "最大瞬間風風向(360degree)", "最大瞬間風風速時間(LST)", "降水量(mm)", "降水時數(hour)", "最大十分鐘降水量(mm)", "最大十分鐘降水量起始時間(LST)", "最大六十分鐘降水量(mm)", "最大六十分鐘降水量起始時間(LST)", "日照時數(hour)", "日照率(%)", "全天空日射量(MJ/㎡)", "能見度(km)", "A型蒸發量(mm)", "日最高紫外線指數", "日最高紫外線指數時間(LST)", "總雲量(0~10)", "地溫0cm", "地溫5cm", "地溫10cm", "地溫20cm", "地溫30cm", "地溫50cm", "地溫100cm"]

for i in range(2000,2024):
    print(i)
    folder_path = r'C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\weather_data\yilan'
    folder_path += '\\'+ str(i)
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, skiprows=0, header=1)
        df.columns = column_names
        df = df.drop(0)
        temp = pd.concat([merged_data, df])
        merged_data = temp


Max_10_mintes_fix_rainfall_data(merged_data)
Max_60_mintes_fix_rainfall_data(merged_data)
A_type_evaporation(merged_data)
All_sky_insolation(merged_data)
highest_UV_index_of_the_day(merged_data)
merged_data.to_csv('merged_yilan_weather_data.csv', index=False)

print("合併完成！")

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
合併完成！


In [13]:
# chiayi
merged_data = pd.DataFrame()
column_names = ["觀測時間(day)", "測站氣壓(hPa)", "海平面氣壓(hPa)", "測站最高氣壓(hPa)", "測站最高氣壓時間(LST)", "測站最低氣壓(hPa)", "測站最低氣壓時間(LST)", "氣溫(℃)", "最高氣溫(℃)", "最高氣溫時間(LST)", "最低氣溫(℃)", "最低氣溫時間(LST)", "露點溫度(℃)", "相對溼度(%)", "最小相對溼度(%)", "最小相對溼度時間(LST)", "風速(m/s)", "風向(360degree)", "最大瞬間風(m/s)", "最大瞬間風風向(360degree)", "最大瞬間風風速時間(LST)", "降水量(mm)", "降水時數(hour)", "最大十分鐘降水量(mm)", "最大十分鐘降水量起始時間(LST)", "最大六十分鐘降水量(mm)", "最大六十分鐘降水量起始時間(LST)", "日照時數(hour)", "日照率(%)", "全天空日射量(MJ/㎡)", "能見度(km)", "A型蒸發量(mm)", "日最高紫外線指數", "日最高紫外線指數時間(LST)", "總雲量(0~10)", "地溫0cm", "地溫5cm", "地溫10cm", "地溫20cm", "地溫30cm", "地溫50cm", "地溫100cm"]

for i in range(2000,2024):
    print(i)
    folder_path = r'C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\weather_data\chiayi'
    folder_path += '\\'+ str(i)
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, skiprows=0, header=1)
        if(df.columns.size != 42):
            df = df.drop(0)
            print(df.shape)
            temp = pd.concat([merged_data, df] ,join='outer')
            merged_data = temp
            print(merged_data.shape)
        else:
            df.columns = column_names
            df = df.drop(0)
            merged_data = pd.concat([merged_data, df])
            print(merged_data.shape)
            
Max_10_mintes_fix_rainfall_data(merged_data)
Max_60_mintes_fix_rainfall_data(merged_data)
A_type_evaporation(merged_data)
All_sky_insolation(merged_data)
highest_UV_index_of_the_day(merged_data)
merged_data.to_csv('merged_yilan_weather_data.csv', index=False)

print("合併完成！") 

2000
(31, 42)
(62, 42)
(92, 42)
(123, 42)
(152, 42)
(183, 42)
(213, 42)
(244, 42)
(274, 42)
(305, 42)
(336, 42)
(366, 42)
2001
(397, 42)
(428, 42)
(458, 42)
(489, 42)
(517, 42)
(548, 42)
(578, 42)
(609, 42)
(639, 42)
(670, 42)
(701, 42)
(731, 42)
2002
(762, 42)
(793, 42)
(823, 42)
(854, 42)
(882, 42)
(913, 42)
(943, 42)
(974, 42)
(1004, 42)
(1035, 42)
(1066, 42)
(1096, 42)
2003
(1127, 42)
(1158, 42)
(1188, 42)
(31, 40)
(1219, 42)
(1247, 42)
(1278, 42)
(1308, 42)
(1339, 42)
(1369, 42)
(1400, 42)
(1431, 42)
(1461, 42)
2004
(1492, 42)
(1523, 42)
(30, 40)
(1553, 42)
(1584, 42)
(1613, 42)
(1644, 42)
(1674, 42)
(1705, 42)
(1735, 42)
(1766, 42)
(1797, 42)
(1827, 42)
2005
(1858, 42)
(1889, 42)
(1919, 42)
(1950, 42)
(1978, 42)
(2009, 42)
(2039, 42)
(2070, 42)
(2100, 42)
(2131, 42)
(2162, 42)
(2192, 42)
2006
(2223, 42)
(2254, 42)
(2284, 42)
(2315, 42)
(2343, 42)
(2374, 42)
(2404, 42)
(2435, 42)
(2465, 42)
(2496, 42)
(2527, 42)
(2557, 42)
2007
(2588, 42)
(2619, 42)
(2649, 42)
(2680, 42)
(2708, 42

In [14]:
# nantou
merged_data = pd.DataFrame()
column_names = ["觀測時間(day)", "測站氣壓(hPa)", "海平面氣壓(hPa)", "測站最高氣壓(hPa)", "測站最高氣壓時間(LST)", "測站最低氣壓(hPa)", "測站最低氣壓時間(LST)", "氣溫(℃)", "最高氣溫(℃)", "最高氣溫時間(LST)", "最低氣溫(℃)", "最低氣溫時間(LST)", "露點溫度(℃)", "相對溼度(%)", "最小相對溼度(%)", "最小相對溼度時間(LST)", "風速(m/s)", "風向(360degree)", "最大瞬間風(m/s)", "最大瞬間風風向(360degree)", "最大瞬間風風速時間(LST)", "降水量(mm)", "降水時數(hour)", "最大十分鐘降水量(mm)", "最大十分鐘降水量起始時間(LST)", "最大六十分鐘降水量(mm)", "最大六十分鐘降水量起始時間(LST)", "日照時數(hour)", "日照率(%)", "全天空日射量(MJ/㎡)", "能見度(km)", "A型蒸發量(mm)", "日最高紫外線指數", "日最高紫外線指數時間(LST)", "總雲量(0~10)", "地溫0cm", "地溫5cm", "地溫10cm", "地溫20cm", "地溫30cm", "地溫50cm", "地溫100cm"]

for i in range(2000,2024):
    print(i)
    folder_path = r'C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\weather_data\nantou'
    folder_path += '\\'+ str(i)
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, skiprows=0, header=1)
        if(df.columns.size != 42):
            df = df.drop(0)
            temp = pd.concat([merged_data, df] ,join='outer')
            merged_data = temp
        else:
            df.columns = column_names
            df = df.drop(0)
            merged_data = pd.concat([merged_data, df])
            
Max_10_mintes_fix_rainfall_data(merged_data)
Max_60_mintes_fix_rainfall_data(merged_data)
A_type_evaporation(merged_data)
All_sky_insolation(merged_data)
highest_UV_index_of_the_day(merged_data)
merged_data.to_csv('merged_yilan_weather_data.csv', index=False)

print("合併完成！") 

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
合併完成！


In [15]:
# taichung
merged_data = pd.DataFrame()
column_names = ["觀測時間(day)", "測站氣壓(hPa)", "海平面氣壓(hPa)", "測站最高氣壓(hPa)", "測站最高氣壓時間(LST)", "測站最低氣壓(hPa)", "測站最低氣壓時間(LST)", "氣溫(℃)", "最高氣溫(℃)", "最高氣溫時間(LST)", "最低氣溫(℃)", "最低氣溫時間(LST)", "露點溫度(℃)", "相對溼度(%)", "最小相對溼度(%)", "最小相對溼度時間(LST)", "風速(m/s)", "風向(360degree)", "最大瞬間風(m/s)", "最大瞬間風風向(360degree)", "最大瞬間風風速時間(LST)", "降水量(mm)", "降水時數(hour)", "最大十分鐘降水量(mm)", "最大十分鐘降水量起始時間(LST)", "最大六十分鐘降水量(mm)", "最大六十分鐘降水量起始時間(LST)", "日照時數(hour)", "日照率(%)", "全天空日射量(MJ/㎡)", "能見度(km)", "A型蒸發量(mm)", "日最高紫外線指數", "日最高紫外線指數時間(LST)", "總雲量(0~10)", "地溫0cm", "地溫5cm", "地溫10cm", "地溫20cm", "地溫30cm", "地溫50cm", "地溫100cm"]

for i in range(2000,2024):
    print(i)
    folder_path = r'C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\weather_data\taichung'
    folder_path += '\\'+ str(i)
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, skiprows=0, header=1)
        if(df.columns.size != 42):
            df = df.drop(0)
            temp = pd.concat([merged_data, df] ,join='outer')
            merged_data = temp
        else:
            df.columns = column_names
            df = df.drop(0)
            merged_data = pd.concat([merged_data, df])

Max_10_mintes_fix_rainfall_data(merged_data)
Max_60_mintes_fix_rainfall_data(merged_data)
A_type_evaporation(merged_data)
All_sky_insolation(merged_data)
highest_UV_index_of_the_day(merged_data)
merged_data.to_csv('merged_yilan_weather_data.csv', index=False)

print("合併完成！")

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
合併完成！


In [157]:
df1 = pd.read_csv(r"C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\蔡昇翰\merged_chiayi_weather_data.csv")
df2 = pd.read_csv(r"C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\蔡昇翰\merged_nantou_weather_data.csv")
df3 = pd.read_csv(r"C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\蔡昇翰\merged_taichung_weather_data.csv")
df4 = pd.read_csv(r"C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\蔡昇翰\merged_yilan_weather_data.csv")

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

nan_count_per_column1 = df1['測站最高氣壓時間(LST)'].isna().sum()
nan_count_per_column2 = df2['測站最高氣壓時間(LST)'].isna().sum()
nan_count_per_column3 = df3['測站最高氣壓時間(LST)'].isna().sum()
nan_count_per_column4 = df4['測站最高氣壓時間(LST)'].isna().sum()
print(nan_count_per_column1)
print(nan_count_per_column2)
print(nan_count_per_column3)
print(nan_count_per_column4)

(8764, 42)
(8766, 43)
(8765, 42)
(8749, 42)
0
0
0
0


添加缺失值標記以及對於2016/1/1的日期差

In [12]:
df = pd.read_csv('merged_data.csv')

df['測站最高氣壓時間(LST)'] = pd.to_datetime(df['測站最高氣壓時間(LST)'], format='%Y/%m/%d %H:%M:%S')

# 計算日期與基準日期（2016/01/01）之間的差異，並新增一個 "日期" 欄位
base_date = pd.to_datetime('2000/01/01', format='%Y/%m/%d')
df['dayoffset'] = (df['測站最高氣壓時間(LST)'] - base_date).dt.days+1
df = pd.concat([df['dayoffset'], df.drop(columns=['dayoffset'])], axis=1)
# 顯示處理後的 DataFrame
# 將處理後的 DataFrame 寫入新的 CSV 檔案
df.to_csv('merged_data.csv', index=False)
print(df['dayoffset'])

0       367.0
1       368.0
2       369.0
3       370.0
4       371.0
        ...  
2246      NaN
2247      NaN
2248      NaN
2249      NaN
2250      NaN
Name: dayoffset, Length: 2251, dtype: float64


平均處理跳過缺失值的

In [17]:
a = ['chiayi','nantou','taichung','yilan']
for i in a:
    addr = r"C:\Users\user\OneDrive\桌面\GitHub\Big-Data-project\蔡昇翰\merged_"+ i +"_weather_data.csv"
    df = pd.read_csv(addr)

    df['測站最高氣壓時間(LST)'] = pd.to_datetime(df['測站最高氣壓時間(LST)'], format='%Y/%m/%d %H:%M:%S')

    # 計算日期與基準日期（2016/01/01）之間的差異，並新增一個 "日期" 欄位
    base_date = pd.to_datetime('2000/01/01', format='%Y/%m/%d')
    df['dayoffset'] = (df['測站最高氣壓時間(LST)'] - base_date).dt.days+1
    df = pd.concat([df['dayoffset'], df.drop(columns=['dayoffset'])], axis=1)
    # 顯示處理後的 DataFrame
    # 將處理後的 DataFrame 寫入新的 CSV 檔案
    path = 'ALL_'+ i + '_merged_data.csv'
    df.to_csv(path , index=False)
    print(df['dayoffset'])

0          1
1          2
2          3
3          4
4          5
        ... 
8759    8670
8760    8671
8761    8672
8762    8673
8763    8674
Name: dayoffset, Length: 8764, dtype: int64
0          1
1          2
2          3
3          4
4          5
        ... 
8761    8670
8762    8671
8763    8672
8764    8673
8765    8674
Name: dayoffset, Length: 8766, dtype: int64
0          1
1          2
2          3
3          4
4          5
        ... 
8760    8670
8761    8671
8762    8672
8763    8673
8764    8674
Name: dayoffset, Length: 8765, dtype: int64
0          1
1          2
2          3
3          4
4          5
        ... 
8760    8670
8761    8671
8762    8672
8763    8673
8764    8674
Name: dayoffset, Length: 8765, dtype: int64


In [21]:
features=["測站氣壓(hPa)","海平面氣壓(hPa)","測站最高氣壓(hPa)","測站最低氣壓(hPa)","氣溫(℃)","最高氣溫(℃)","最低氣溫(℃)","露點溫度(℃)","相對溼度(%)","最小相對溼度(%)","風速(m/s)","風向(360degree)","最大瞬間風(m/s)","最大瞬間風風向(360degree)","降水量(mm)","降水時數(hour)","最大十分鐘降水量(mm)","最大六十分鐘降水量(mm)","日照時數(hour)","全天空日射量(MJ/㎡)","能見度(km)","A型蒸發量(mm)","日最高紫外線指數","總雲量(0~10)","地溫0cm","地溫5cm","地溫10cm","地溫20cm","地溫30cm","地溫50cm","地溫100cm"]
new_features = [f'{feature}_avg' for feature in features]
pre_features=["Trans_Quantity_day1","Trans_Quantity_day2","Trans_Quantity_day3","Trans_Quantity_day4","Trans_Quantity_day5","Avg_Price_day1","Avg_Price_day2","Avg_Price_day3","Avg_Price_day4","Avg_Price_day5"]
# Avg_Price
# Trans_Quantity

# 迴圈處理每個檔案


# 讀取 CSV 檔案
df = pd.read_csv('merged_weather_data.csv')

dest = pd.read_csv('new_cabbage.csv')
for day in range(548,2558):##2558 733
    target_row =  dest[dest['dayoffset'] == day]
    index=target_row.index
    if target_row.empty:
        continue
    # 初始化每個檔案的溫度總和和有效數據個數
    selected_data = pd.Series([])
    for i in range(5):
        selected_data = pd.concat([selected_data, dest.loc[index - 5 + i, 'Trans_Quantity']])
    for i in range(5):
        selected_data = pd.concat([selected_data, dest.loc[index - 5 + i, 'Avg_Price']])
    dest.loc[dest['dayoffset'] == day, pre_features] = selected_data.values
    dest.loc[dest['dayoffset'] == day, features] = df.loc[df['dayoffset'] == day, features].values
for day in range (0,548):#548
    target_row = dest[dest['dayoffset'] == day]
    if target_row.empty:
        continue
    dest = dest.drop(dest[dest['dayoffset'] == day].index)
##填原本的值回去

##再來是平均後的值
for day in range(548,2558):##2558 733
    target_row = dest[dest['dayoffset'] == day]
    if target_row.empty:
        continue
    # 初始化每個檔案的溫度總和和有效數據個數
    for temp, new_feature in zip(features, new_features):
        temp_sum = 0
        cnt=0
        valid_count = 0
        
        # average 60 days
        for num in range (day-60,day):
            goal_row = df[df['dayoffset'] == num]
            if goal_row.empty:
                continue
            # 檢查是否為有效數據
            data=goal_row[temp]
            if not data.isin(["--", "T", "X", "/"]).all():
                temp_sum = temp_sum+float(data)
                valid_count = valid_count + 1
        if valid_count==0:
            avg=0
        else:
            avg=temp_sum/valid_count
        dest.loc[dest['dayoffset'] == day,new_feature] = avg

        # find 3 days average tempature "highest" and "lowest"
        if(temp == "氣溫(℃)"):
            #  3 highest tempature  
            max3 = [-1,-1,-1]
            for num in range(day-60,day):
                goal_row = df[df['dayoffset'] == num]
                if goal_row.empty:
                    continue

                data=goal_row[temp]
                if not data.isin(["--", "T", "X", "/"]).all():
                    data=float(data)
                    min_tempature = min(max3)
                    if(data > min_tempature):
                        max3 = [data if x == min_tempature else x for x in max3]
            # add feature
            max3_average = sum(max3)/3
            dest.loc[dest['dayoffset']== day,"3highest_temp_avg"] = max3_average
                
            # 3 lowest tempature 
            min3 = [100,100,100]
            for num in range(day-60,day):
                goal_row = df[df['dayoffset'] == num]
                if goal_row.empty:
                    continue

                data=goal_row[temp]
                if not data.isin(["--", "T", "X", "/"]).all():
                    data=float(data)
                    max_tempature = max(min3)
                    if(data < max_tempature):
                        min3 = [data if x == max_tempature else x for x in min3]
            # add feature 
            min3_average = sum(min3)/3
            dest.loc[dest['dayoffset']== day,"3lowest_temp_avg"] = min3_average

            # days higher than 22 degree and lower than 13
            higher_num = 0
            lower_num = 0 
            for num in range(day-60,day):
                goal_row = df[df['dayoffset'] == num]
                if goal_row.empty:
                    continue
                data=goal_row[temp]
                if not data.isin(["--", "T", "X", "/"]).all():
                    data = float(data)
                    if(data > 22):
                        higher_num = higher_num + 1
                    if(data < 13):
                        lower_num = lower_num + 1

            dest.loc[dest['dayoffset']== day,"TmpHigher22_num"] = higher_num
            dest.loc[dest['dayoffset']== day,"TmpLower13_num"] = lower_num
        
        if(temp == "降水量(mm)"):
            #  3 highest rain fall  
            max3 = [-1,-1,-1]
            for num in range(day-60,day):
                goal_row = df[df['dayoffset'] == num]
                if goal_row.empty:
                    continue

                data=goal_row[temp]
                if not data.isin(["--", "T", "X", "/"]).all():
                    data=float(data)
                    min_rainfall = min(max3)
                    if(data > min_rainfall):
                        max3 = [data if x == min_rainfall else x for x in max3]
            # add feature
            max3_average = sum(max3)/3
            dest.loc[dest['dayoffset']== day,"3highest_RainFall_avg"] = max3_average



dest.to_csv('New_final_training_data.csv', index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_15484\538959614.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  temp_sum = temp_sum+float(data)
C:\Users\user\AppData\Local\Temp\ipykernel_15484\538959614.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  temp_sum = temp_sum+float(data)
C:\Users\user\AppData\Local\Temp\ipykernel_15484\538959614.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  temp_sum = temp_sum+float(data)
C:\Users\user\AppData\Local\Temp\ipykernel_15484\538959614.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  temp_sum = temp_sum+float(data)
C:\Users\user\AppData\Local\